In [1]:
!pip install pandas

In [2]:
!pip install tensorflow

In [3]:
!pip install numpy

In [4]:
import pandas as pd

In [5]:
cancerDataSet = pd.read_csv("BreastCancerWisconsinDataSet.csv")

In [6]:
cancerDataSet.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
y = cancerDataSet['diagnosis'].map(lambda d: 1 if d == 'M' else 0).values
X = cancerDataSet.drop(['diagnosis', 'id'], axis=1).values

In [8]:
X[1]

array([2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
       8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
       3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
       1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
       1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02])

In [9]:
X.shape

(569, 30)

In [10]:
# Partition the input data into a training set and a test set
m = X.shape[0]
m_train = 400
m_test = m - m_train

X_train = X[:m_train]
y_train = y[:m_train]

X_test = X[m_train:]
y_test = y[m_train:]

# Gets a batch of the training data. 
# NOTE: Rather than loading a whole large data set as above and then taking array slices as done here, 
#       This method can connect to a data source and select just the batch needed.
def get_training_batch(batch_num, batch_size):
    lower = batch_num * (m_train // batch_size)
    upper = lower + batch_size
    return X_train[lower:upper], y_train[lower:upper]

## TensorFlow Implementation

In [11]:
import tensorflow as tf
import numpy as np

# Make this notebook's output stable across runs
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

# A method to build a new neural net layer of a given size,  
# fully connect it to a given preceding layer X, and 
# compute its output Z either with or without (default) an activation function
# Call with activation=tf.nn.relu or tf.nn.sigmoid or tf.nn.tanh, for examples

def make_nn_layer(layer_name, layer_size, X, activation=None):
    with tf.name_scope(layer_name):
        X_size = int(X.get_shape()[1])
        SD = 2 / np.sqrt(X_size)
        weights = tf.truncated_normal((X_size, layer_size), dtype=tf.float64, stddev=SD)
        W = tf.Variable(weights, name='weights')
        b = tf.Variable(tf.zeros([layer_size], dtype=tf.float64), name='biases')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [12]:
# Make the neural net structure
n = int(X.shape[1])
print(n)
n_inputs = n
n_hidden1 = 10
n_hidden2 = 5
n_outputs = 2   # Two output classes: malignant or non-malignant

X = tf.placeholder(tf.float64, shape=(None, n_inputs), name='X')

with tf.name_scope('nn'):
    hidden1 = make_nn_layer('hidden1', n_hidden1, X, activation=tf.nn.relu)
    hidden2 = make_nn_layer('hidden2', n_hidden2, hidden1, activation=tf.nn.relu)
    outputs = make_nn_layer('outputs', n_outputs, hidden2) 
    outputs = tf.identity(outputs, "nn_output")
    
y = tf.placeholder(tf.int64, shape=(None), name='y')

30
Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# Define how the neural net will learn

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=outputs)
    loss = tf.reduce_mean(xentropy, name='l')
    
learning_rate = 0.0001
with tf.name_scope("train"):
#    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("test"):
    correct = tf.nn.in_top_k(tf.cast(outputs, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    

In [14]:
# Set up the ability to save and restore the trained neural net...
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [15]:
# TRAINING TIME

# This is how many times to use the full set of training data
n_epochs = 1000

# For a larger training set, it's typically necessary to break training into
# batches so only the memory needed to store one batch of training data is used
batch_size = 50

with tf.Session() as training_session:
    init.run()
    
    for epoch in range(n_epochs):
        
        # Loop through the whole training set in batches
        for batch_num in range(m_train // batch_size):
            X_batch, y_batch = get_training_batch(batch_num, batch_size)
            training_session.run(training_op, feed_dict={X: X_batch, y: y_batch})

        if epoch % 100 == 99:
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch+1, "Training accuracy:", acc_train, "Testing accuracy:", acc_test)

    save_path = saver.save(training_session, ".\CancerNN")
    
    # A quick test with the trained model 
    Z = outputs.eval(feed_dict={X: X_test[:40]})
    y_pred = np.argmax(Z, axis=1)
    print("")
    print("Actual classes:   ", y_test[:40])  
    print("Predicted classes:", y_pred)

100 Training accuracy: 0.4325 Testing accuracy: 0.23076923
200 Training accuracy: 0.8725 Testing accuracy: 0.7751479
300 Training accuracy: 0.87 Testing accuracy: 0.7751479
400 Training accuracy: 0.875 Testing accuracy: 0.7751479
500 Training accuracy: 0.875 Testing accuracy: 0.7751479
600 Training accuracy: 0.88 Testing accuracy: 0.7810651
700 Training accuracy: 0.875 Testing accuracy: 0.78698224
800 Training accuracy: 0.875 Testing accuracy: 0.8165681
900 Training accuracy: 0.885 Testing accuracy: 0.83431953
1000 Training accuracy: 0.8875 Testing accuracy: 0.84615386

Actual classes:    [1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0
 0 0 0]
Predicted classes: [1 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0
 1 1 0]


## Keras Implementation

In [16]:
from keras.models import Model
from keras.layers import Activation, Dense, Dropout, Input
from keras.optimizers import Adam, RMSprop
from numpy.random import seed
seed(1)

ModuleNotFoundError: No module named 'keras'

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_train[0])
print(y_train[0])
print(X_train[5])
print(y_train[50])

In [ ]:
batch_size = 100

Inputs = Input(shape=(30,))
Dense_1 = Dense(30)(Inputs)
Dense_2 = Dense(30, activation="relu")(Dense_1)
Dense_3 = Dense(30, activation="relu")(Dense_2)
Dense_4 = Dense(2, activation="linear")(Dense_3)
Classifier = Activation('softmax')(Dense_4)

model = Model(input=Inputs, output=Classifier)
model.summary()

#optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
optimizer = RMSprop(lr=0.0001)
    
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
 
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

In [ ]:
score = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])